In [10]:
import streamlit as st
import pandas as pd

import local_functions as lf
from threading import Thread
from math import floor
import plotly.graph_objects as go
import plotly.subplots as pls
import pandas as pd
import numpy as np
import pydeck as pdk

########################################################################################
##################### Page Configuration ###############################################
########################################################################################

#Page layout and quick title
st.set_page_config(
    page_title="Calgary Traffic",
    page_icon="🚗",
    layout="wide",
    initial_sidebar_state="expanded")

#Page header
st.header(f"Traffic in Calgary", divider = True)

#Split into columns
col1, col2 = st.columns([0.2, 0.8])

########################################################################################
##################### Data Wrangle #####################################################
########################################################################################

chart_data = pd.read_csv('traffic_detection-2.csv')

chart_data.rename(columns = {'camera_location': 'Intersection'}, inplace = True)
chart_data['traffic_sum'] = chart_data[['car','truck','bus']].sum(axis = 1)
chart_data['Traffic'] = pd.cut(chart_data['traffic_sum'], 
                            bins = chart_data['traffic_sum'].quantile([0,.3, .7,1]),
                            labels = ['Low', 'Moderate', 'High'])

#Colour index for map (below)
min_count = min(chart_data['traffic_sum'])
max_count = max(chart_data['traffic_sum'])

chart_data['color_column'] = chart_data.apply(lf.get_color, min = min_count, max = max_count, axis = 1)


In [17]:
def get_sideplot(chart_data):
    
    labels = list(chart_data['Traffic'].dropna().unique())
    values = list(chart_data['Traffic'].value_counts())


    fig = pls.make_subplots(rows=2, cols=1, specs=[[{'type':'domain'}],[{'type':'xy'}]], row_width=[0.3, 0.7])
    fig.add_trace(go.Pie(labels=labels, values=values, title=f"Traffic <br> <b>Levels</b>", 
                        title_font=dict(size=20, color = "white", family='Arial, sans-serif'),
                        hole=.4, hoverinfo="label+value+name", textinfo='percent+label',
                        marker=dict(colors=['darkorange', 'lightyellow', 'darkred'], line=dict(color='#000000', width=2))), 
                    row=1, col=1) 
    n=16 #example hours
    x_example = np.arange(n*12)
    fig.add_trace(go.Scatter(
        x = pd.date_range('2024-04-01', periods=24*12, freq='5min'),
        y = np.sin(1/2*np.pi*x_example/n)+np.cos(1/4*np.pi*x_example/n) + 100,
        marker=dict(color='grey'),
        name="Historical Traffic",
        mode = 'lines'
    ), row=2, col=1)

    fig.update_layout(
        title_text="PE",
        showlegend=False,
        margin={"r": 0, "t": 0, "l": 0, "b": 0}, plot_bgcolor='#0e1117', paper_bgcolor='#0e1117', autosize = True)

    fig.update_yaxes(title_text="Avg. Traffic Vol.", row=2, col=1, showgrid=False)
    fig.update_xaxes(title_text="Time", row=2, col=1, showgrid=False)
    global sideplot2
    sideplot2 = fig

def get_map(chart_data):
    fig = pdk.Deck(
            map_style=None,
            initial_view_state=pdk.ViewState(
                latitude=51.066886,
                longitude=-114.065353,
                zoom=10,
                pitch=50,
            ),
            layers=[
                pdk.Layer(
                'ColumnLayer',
                data=chart_data,
                get_position='[longitude, latitude]',
                get_elevation="traffic_sum",
                radius=200,
                elevation_scale=100,
                get_fill_color='color_column',
                elevation_range=[0, 1000],
                pickable=True,
                extruded=True,
                )
            ],
        )
    global map
    map = fig

In [11]:
thread_sideplot = Thread(target=get_sideplot, args=[chart_data])
thread_map = Thread(target=get_map, args=[chart_data])

sideplot = thread_sideplot.start()
map = thread_map.start()

In [18]:
thread_sideplot = Thread(target=get_sideplot, args=[chart_data])
thread_sideplot.start()

In [19]:
sideplot